In [ ]:
import tensorflow.keras
import tensorflow as tf

from tensorflow.keras.applications import EfficientNetB0,MobileNet  
#from tensorflow.keras.applications import MobilNet
from tensorflow.keras.models import Model
import numpy as np
import random
import time
from numpy.linalg import norm
import PIL
from PIL import Image
import glob
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.layers import GlobalAveragePooling2D, Concatenate

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D,Dense
embedding_size = 1024

# Load pre-trained SqueezeNet model without top (fully connected) layers
base_model = tf.keras.applications.MobileNet(weights='imagenet', include_top=False, input_shape = (224,224,3))

#Remove the last layer of MobileNet
base_model_output = base_model.layers[-1].output

x = base_model.output

# Step 3: Adjust the architecture for retrieval
x = GlobalAveragePooling2D()(x)

# Add the final Dense layer with embedding_size
embeddings = Dense(embedding_size)(x)

# Add L2 normalization layer


class L2Normalization(tf.keras.layers.Layer):
    def __init__(self, axis=-1, epsilon=1e-12, **kwargs):
        super(L2Normalization, self).__init__(**kwargs)
        self.axis = axis
        self.epsilon = epsilon

    def call(self, inputs):
        return tf.math.l2_normalize(inputs, axis=self.axis, epsilon=self.epsilon)
    
    def get_config(self):
        config = super(L2Normalization, self).get_config()
        config.update({'axis': self.axis, 'epsilon': self.epsilon})
        return config

    # Apply L2 normalization
    
embeddings = tf.keras.layers.Dense(embedding_size)(x)
embeddings = L2Normalization()(embeddings)


feature_extraction_model1 = Model(inputs=base_model.input, outputs=embeddings)

feature_extraction_model1.summary()

In [ ]:
# Load the base model
base_model2 = EfficientNetB0(weights='imagenet', include_top=False,input_shape = (224,224,3))
base_model2.summary()

In [ ]:
# Now define the layers for the second parallel network
# You can define them similar to the first one
layer4 = base_model2.get_layer('block2a_expand_activation').output
layer5 = base_model2.get_layer('block4a_expand_activation').output
layer6 = base_model2.get_layer('block6a_expand_activation').output
# Apply Global Average Pooling to each feature from the third layer
layer4 = GlobalAveragePooling2D()(layer4)

# Apply Global Average Pooling to each feature from the fourth layer
layer5 = GlobalAveragePooling2D()(layer5)


layer6 = GlobalAveragePooling2D()(layer6)

# Concatenate the feature vectors from the second set of layers
concatenated_features2 = Concatenate()([layer4, layer5, layer6])

# Create a model for feature extraction
feature_extraction_model2 = Model(inputs= base_model2.input, outputs= concatenated_features2)


feature_extraction_model2.summary()

In [ ]:
#Create list of files of dataset
extensions = extensions = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG', '.bmp', '.BMP']

def get_file_list(root_dir):
    file_list = []
    counter = 1
    for root, directories, filenames in os.walk(root_dir):
        for filename in filenames:
            if any(ext in filename for ext in extensions):
                file_list.append(os.path.join(root, filename))
                counter += 1
    return file_list

In [ ]:
#obtain file names 
import os
# path to the datasets
root_dir = 'C:\\Data Drive\\Datasets\\101_ObjectCategories-20230908T025813Z-001\\101_ObjectCategories'
filenames = sorted(get_file_list(root_dir))

In [ ]:
#assigning labels to dataset
import pathlib
data_dir = pathlib.Path(root_dir)
data_dir
dataset_labels=[]

object_categories_dict = {
     #'BACKGROUND_Google': list(data_dir.glob('BACKGROUND_Google/*')),
     'Faces': list(data_dir.glob('Faces/*')),
     'Faces_easy': list(data_dir.glob('Faces_easy/*')),
     'Leopards': list(data_dir.glob('Leopards/*')),
     'Motorbikes': list(data_dir.glob('Motorbikes/*')),
     'accordion': list(data_dir.glob('accordion/*')),
     'airplanes': list(data_dir.glob('airplanes/*')),
     'anchor': list(data_dir.glob('anchor/*')),
     'ant': list(data_dir.glob('ant/*')),
     'barrel': list(data_dir.glob('barrel/*')),
     'bass': list(data_dir.glob('bass/*')),
     'beaver': list(data_dir.glob('beaver/*')),
     'binocular': list(data_dir.glob('binocular/*')),
     'bonsai': list(data_dir.glob('bonsai/*')),
     'brain': list(data_dir.glob('brain/*')),
     'brontosaurus': list(data_dir.glob('brontosaurus/*')),
     'buddha': list(data_dir.glob('buddha/*')),
     'butterfly': list(data_dir.glob('butterfly/*')),
     'camera': list(data_dir.glob('camera/*')),
     'cannon': list(data_dir.glob('cannon/*')),
     'car_side': list(data_dir.glob('car_side/*')),
     'ceiling_fan': list(data_dir.glob('ceiling_fan/*')),
     'cellphone': list(data_dir.glob('cellphone/*')),
     'chair': list(data_dir.glob('chair/*')),
     'chandelier': list(data_dir.glob('chandelier/*')),
     'cougar_body': list(data_dir.glob('cougar_body/*')),
     'cougar_face': list(data_dir.glob('cougar_face/*')),
     'crab': list(data_dir.glob('crab/*')),
     'crayfish': list(data_dir.glob('crayfish/*')),
     'crocodile_head': list(data_dir.glob('crocodile_head/*')),
     'cup': list(data_dir.glob('cup/*')),
     'dalmatian': list(data_dir.glob('dalmatian/*')),
     'dollar_bill': list(data_dir.glob('dollar_bill/*')),
     'dolphin': list(data_dir.glob('dolphin/*')),
     'dragonfly': list(data_dir.glob('dragonfly/*')),
     'electric_guitar': list(data_dir.glob('electric_guitar/*')),
     'elephant': list(data_dir.glob('elephant/*')),
     'emu': list(data_dir.glob('emu/*')),
     'euphonium': list(data_dir.glob('euphonium/*')),
     'ewer': list(data_dir.glob('ewer/*')),
     'ferry': list(data_dir.glob('ferry/*')),
     'flamingo': list(data_dir.glob('flamingo/*')),
     'flamingo_head': list(data_dir.glob('flamingo_head/*')),
     'garfield': list(data_dir.glob('garfield/*')),
     'gerenuk': list(data_dir.glob('gerenuk/*')),
     'gramophone': list(data_dir.glob('gramophone/*')),
     'grand_piano': list(data_dir.glob('grand_piano/*')),
     'hawksbill': list(data_dir.glob('hawksbill/*')),
     'headphone': list(data_dir.glob('headphone/*')),
     'hedgehog': list(data_dir.glob('hedgehog/*')),
     'helicopter': list(data_dir.glob('helicopter/*')),
     'ibis': list(data_dir.glob('ibis/*')),
     'inline_skate': list(data_dir.glob('inline_skate/*')),
     'joshua_tree': list(data_dir.glob('joshua_tree/*')),
     'kangaroo': list(data_dir.glob('kangaroo/*')),
     'ketch': list(data_dir.glob('ketch/*')),
     'lamp': list(data_dir.glob('lamp/*')),
     'laptop': list(data_dir.glob('laptop/*')),
     'llama': list(data_dir.glob('llama/*')),
     'lobster': list(data_dir.glob('lobster/*')),
     'lotus': list(data_dir.glob('lotus/*')),
     'mandolin': list(data_dir.glob('mandolin/*')),
     'mayfly': list(data_dir.glob('mayfly/*')),
     'menorah': list(data_dir.glob('menorah/*')),
     'metronome': list(data_dir.glob('metronome/*')),
     'minaret': list(data_dir.glob('minaret/*')),
     'nautilus': list(data_dir.glob('nautilus/*')),
     'octopus': list(data_dir.glob('octopus/*')),
     'okapi': list(data_dir.glob('okapi/*')),
     'pagoda': list(data_dir.glob('pagoda/*')),
     'panda': list(data_dir.glob('panda/*')),
     'pigeon': list(data_dir.glob('pigeon/*')),
     'pizza': list(data_dir.glob('pizza/*')),
     'platypus': list(data_dir.glob('platypus/*')),
     'pyramid': list(data_dir.glob('pyramid/*')),
     'revolver': list(data_dir.glob('revolver/*')),
     'rhino': list(data_dir.glob('rhino/*')),
     'rooster': list(data_dir.glob('rooster/*')),
     'saxophone': list(data_dir.glob('saxophone/*')),
     'schooner': list(data_dir.glob('schooner/*')),
     'scissors': list(data_dir.glob('scissors/*')),
     'scorpion': list(data_dir.glob('scorpion/*')),
     'sea_horse': list(data_dir.glob('sea_horse/*')),
     'snoopy': list(data_dir.glob('snoopy/*')),
     'soccer_ball': list(data_dir.glob('soccer_ball/*')),
     'stapler': list(data_dir.glob('stapler/*')),
     'starfish': list(data_dir.glob('starfish/*')),
     'stegosaurus': list(data_dir.glob('stegosaurus/*')),
     'stop_sign': list(data_dir.glob('stop_sign/*')),
     'strawberry': list(data_dir.glob('strawberry/*')),
     'sunflower': list(data_dir.glob('sunflower/*')),
     'tick': list(data_dir.glob('tick/*')),
     'umbrella': list(data_dir.glob('umbrella/*')),
     'watch': list(data_dir.glob('watch/*')),
     'water_lilly': list(data_dir.glob('water_lilly/*')),
     'wheelchair': list(data_dir.glob('wheelchair/*')),
     'wild_cat': list(data_dir.glob('wild_cat/*')),
     'windsor_chair': list(data_dir.glob('windsor_chair/*')),
     'wrench': list(data_dir.glob('wrench/*')),
     'yin_yang': list(data_dir.glob('yin_yang/*')),
     'crocodile': list(data_dir.glob('crocodile/*')),
     'trilobite': list(data_dir.glob('trilobite/*'))
}
object_labels_dict = {
    #'BACKGROUND_Google': 0,
     'Faces': 1,
     'Faces_easy': 2,
     'Leopards': 3,
     'Motorbikes': 4,
     'accordion': 5,
     'airplanes': 6,
     'anchor': 7,
     'ant': 8,
     'barrel': 9,
     'bass': 10,
     'beaver': 11,
     'binocular': 12,
     'bonsai': 13,
     'brain': 14,
     'brontosaurus': 15,
     'buddha': 16,
     'butterfly': 17,
     'camera': 18,
     'cannon': 19,
     'car_side': 20,
     'ceiling_fan': 21,
     'cellphone': 22,
     'chair': 23,
     'chandelier': 24,
     'cougar_body': 25,
     'cougar_face': 26,
     'crab': 27,
     'crayfish': 28,
     'crocodile_head': 29,
     'cup': 30,
     'dalmatian': 31,
     'dollar_bill': 32,
     'dolphin': 33,
     'dragonfly': 34,
     'electric_guitar': 35,
     'elephant': 36,
     'emu': 37,
     'euphonium': 38,
     'ewer': 39,
     'ferry': 40,
     'flamingo': 41,
     'flamingo_head': 42,
     'garfield': 43,
     'gerenuk': 44,
     'gramophone': 45,
     'grand_piano': 46,
     'hawksbill': 47,
     'headphone': 48,
     'hedgehog': 49,
     'helicopter': 50,
     'ibis': 51,
     'inline_skate': 52,
     'joshua_tree': 53,
     'kangaroo': 54,
     'ketch': 55,
     'lamp': 56,
     'laptop': 57,
     'llama': 58,
     'lobster': 59,
     'lotus': 60,
     'mandolin': 61,
     'mayfly': 62,
     'menorah': 63,
     'metronome': 64,
     'minaret': 65,
     'nautilus': 66,
     'octopus': 67,
     'okapi': 68,
     'pagoda': 69,
     'panda': 70,
     'pigeon': 71,
     'pizza':72,
     'platypus': 73,
     'pyramid': 74,
     'revolver': 75,
     'rhino': 76,
     'rooster': 77,
     'saxophone': 78,
     'schooner': 79,
     'scissors': 80,
     'scorpion': 81,
     'sea_horse': 82,
     'snoopy': 83,
     'soccer_ball': 84,
     'stapler': 85,
     'starfish': 86,
     'stegosaurus': 87,
     'stop_sign': 88,
     'strawberry': 89,
     'sunflower': 90,
     'tick': 91,
     'umbrella': 92,
     'watch': 93,
     'water_lilly': 94,
     'wheelchair': 95,
     'wild_cat': 96,
     'windsor_chair': 97,
     'wrench': 98,
     'yin_yang': 99,
     'crocodile':100,
     'trilobite':101,   
}
#class_to_numeric = {class_label: i for i, class_label in enumerate(set(class_labels))}
for object_category,images in object_categories_dict.items():
    for image in images:
        dataset_labels.append(object_labels_dict[object_category])


In [ ]:
len(filenames)

In [ ]:
import cv2
X, y = [], []
for object_category,filenames in object_categories_dict.items():
    for filename in filenames:
        img = cv2.imread(str(filename))  # Convert Path object to string
        resized_img = cv2.resize(img, (224, 224))
        X.append(resized_img)
        y.append(object_labels_dict[object_category])

In [ ]:
X = np.array(X) 
y = np.array(y)

In [ ]:
len(y)

In [ ]:

x = X / 255.0  # Normalize pixel values to the range [0, 1]


In [ ]:
print("x dimensions:", x.shape)
print("y dimensions:", y.shape)

In [ ]:
import timeit


start_time = timeit.default_timer()
features_model1 = feature_extraction_model1.predict(x)
end_time = timeit.default_timer()
elapsed_time = end_time - start_time  # Calculate the elapsed time
print(f"Elaapsed Time: {elapsed_time:.4f} seconds")

In [ ]:
import timeit
start_time = timeit.default_timer()
features_model2 = feature_extraction_model2.predict(x)
end_time = timeit.default_timer()
elapsed_time = end_time - start_time  # Calculate the elapsed time
print(f"Elaapsed Time: {elapsed_time:.4f} seconds")

In [ ]:
# Concatenate both concatenated feature vectors
extracted_features = Concatenate()([features_model1,features_model2])


In [ ]:
# Assign labels to the extracted features
features_with_labels = list(zip(extracted_features, y))

In [ ]:


# Example usage: Iterating over features with their labels
for feature, label in features_with_labels:
    # Access the feature vector and its corresponding label
    print("Feature shape:", feature.shape)
    print("Label:", label)
    # Perform any operations using the feature and label

In [ ]:
# Reshape the extracted features array
features = tf.reshape(extracted_features, (len(extracted_features), -1))
print("Reshaped features shape:", features.shape)


In [ ]:
# Assuming features is your tensor
num_images, num_features_per_image = features.shape

# Printing the values
print("Number of images =", num_images)
print("Number of features per image =", num_features_per_image)

In [ ]:
 #Assuming 'features' is a NumPy array containing your extracted features
np.save('extracted_features_using_fusion_caltech101_effmobilnet.npy', features)

# Reading and extracting features of query images 

In [ ]:
#obtain file names 
import os
# path to the datasets
root_dir_query_images= 'C:\\Data Drive\\Datasets\\101_ObjectCategories-20230908T025813Z-001\\test'
filenames_query_images = sorted(get_file_list(root_dir_query_images))

In [ ]:
filenames_query_images[:5] 

In [ ]:
num_query_images = len(filenames_query_images)

num_query_images

In [ ]:
#assigning labels to dataset
import pathlib
data_dir_query = pathlib.Path(root_dir_query_images)
#data_dir_query
dataset_labels_query=[]

object_categories_dict_q = {
    #'BACKGROUND_Google': list(data_dir.glob('BACKGROUND_Google/*')),
     'Faces': list(data_dir_query.glob('Faces/*')),
    'Faces_easy': list(data_dir_query.glob('Faces_easy/*')),
    'Leopards': list(data_dir_query.glob('Leopards/*')),
    'Motorbikes': list(data_dir_query.glob('Motorbikes/*')),
    'accordion': list(data_dir_query.glob('accordion/*')),
    'airplanes': list(data_dir_query.glob('airplanes/*')),
    'anchor': list(data_dir_query.glob('anchor/*')),
    'ant': list(data_dir_query.glob('ant/*')),
    'barrel': list(data_dir_query.glob('barrel/*')),
    'bass': list(data_dir_query.glob('bass/*')),
    'beaver': list(data_dir_query.glob('beaver/*')),
    'binocular': list(data_dir_query.glob('binocular/*')),
    'bonsai': list(data_dir_query.glob('bonsai/*')),
    'brain': list(data_dir_query.glob('brain/*')),
    'brontosaurus': list(data_dir_query.glob('brontosaurus/*')),
    'buddha': list(data_dir_query.glob('buddha/*')),
    'butterfly': list(data_dir_query.glob('butterfly/*')),
    'camera': list(data_dir_query.glob('camera/*')),
    'cannon': list(data_dir_query.glob('cannon/*')),
    'car_side': list(data_dir_query.glob('car_side/*')),
    'ceiling_fan': list(data_dir_query.glob('ceiling_fan/*')),
    'cellphone': list(data_dir_query.glob('cellphone/*')),
    'chair': list(data_dir_query.glob('chair/*')),
    'chandelier': list(data_dir_query.glob('chandelier/*')),
    'cougar_body': list(data_dir_query.glob('cougar_body/*')),
    'cougar_face': list(data_dir_query.glob('cougar_face/*')),
    'crab': list(data_dir_query.glob('crab/*')),
    'crayfish': list(data_dir_query.glob('crayfish/*')),
    'crocodile_head': list(data_dir_query.glob('crocodile_head/*')),
    'cup': list(data_dir_query.glob('cup/*')),
    'dalmatian': list(data_dir_query.glob('dalmatian/*')),
    'dollar_bill': list(data_dir_query.glob('dollar_bill/*')),
    'dolphin': list(data_dir_query.glob('dolphin/*')),
    'dragonfly': list(data_dir_query.glob('dragonfly/*')),
    'electric_guitar': list(data_dir_query.glob('electric_guitar/*')),
    'elephant': list(data_dir_query.glob('elephant/*')),
    'emu': list(data_dir_query.glob('emu/*')),
    'euphonium': list(data_dir_query.glob('euphonium/*')),
    'ewer': list(data_dir_query.glob('ewer/*')),
    'ferry': list(data_dir_query.glob('ferry/*')),
    'flamingo': list(data_dir_query.glob('flamingo/*')),
    'flamingo_head': list(data_dir_query.glob('flamingo_head/*')),
    'garfield': list(data_dir_query.glob('garfield/*')),
    'gerenuk': list(data_dir_query.glob('gerenuk/*')),
    'gramophone': list(data_dir_query.glob('gramophone/*')),
    'grand_piano': list(data_dir_query.glob('grand_piano/*')),
    'hawksbill': list(data_dir_query.glob('hawksbill/*')),
    'headphone': list(data_dir_query.glob('headphone/*')),
    'hedgehog': list(data_dir_query.glob('hedgehog/*')),
    'helicopter': list(data_dir_query.glob('helicopter/*')),
    'ibis': list(data_dir_query.glob('ibis/*')),
    'inline_skate': list(data_dir_query.glob('inline_skate/*')),
    'joshua_tree': list(data_dir_query.glob('joshua_tree/*')),
    'kangaroo': list(data_dir_query.glob('kangaroo/*')),
    'ketch': list(data_dir_query.glob('ketch/*')),
    'lamp': list(data_dir_query.glob('lamp/*')),
    'laptop': list(data_dir_query.glob('laptop/*')),
    'llama': list(data_dir_query.glob('llama/*')),
    'lobster': list(data_dir_query.glob('lobster/*')),
    'lotus': list(data_dir_query.glob('lotus/*')),
    'mandolin': list(data_dir_query.glob('mandolin/*')),
    'mayfly': list(data_dir_query.glob('mayfly/*')),
    'menorah': list(data_dir_query.glob('menorah/*')),
    'metronome': list(data_dir_query.glob('metronome/*')),
    'minaret': list(data_dir_query.glob('minaret/*')),
    'nautilus': list(data_dir_query.glob('nautilus/*')),
    'octopus': list(data_dir_query.glob('octopus/*')),
    'okapi': list(data_dir_query.glob('okapi/*')),
    'pagoda': list(data_dir_query.glob('pagoda/*')),
    'panda': list(data_dir_query.glob('panda/*')),
    'pigeon': list(data_dir_query.glob('pigeon/*')),
    'pizza': list(data_dir_query.glob('pizza/*')),
    'platypus': list(data_dir_query.glob('platypus/*')),
    'pyramid': list(data_dir_query.glob('pyramid/*')),
    'revolver': list(data_dir_query.glob('revolver/*')),
    'rhino': list(data_dir_query.glob('rhino/*')),
    'rooster': list(data_dir_query.glob('rooster/*')),
    'saxophone': list(data_dir_query.glob('saxophone/*')),
    'schooner': list(data_dir_query.glob('schooner/*')),
    'scissors': list(data_dir_query.glob('scissors/*')),
    'scorpion': list(data_dir_query.glob('scorpion/*')),
    'sea_horse': list(data_dir_query.glob('sea_horse/*')),
    'snoopy': list(data_dir_query.glob('snoopy/*')),
    'soccer_ball': list(data_dir_query.glob('soccer_ball/*')),
    'stapler': list(data_dir_query.glob('stapler/*')),
    'starfish': list(data_dir_query.glob('starfish/*')),
    'stegosaurus': list(data_dir_query.glob('stegosaurus/*')),
    'stop_sign': list(data_dir_query.glob('stop_sign/*')),
    'strawberry': list(data_dir_query.glob('strawberry/*')),
    'sunflower': list(data_dir_query.glob('sunflower/*')),
    'tick': list(data_dir_query.glob('tick/*')),
    'umbrella': list(data_dir_query.glob('umbrella/*')),
    'watch': list(data_dir_query.glob('watch/*')),
    'water_lilly': list(data_dir_query.glob('water_lilly/*')),
    'wheelchair': list(data_dir_query.glob('wheelchair/*')),
    'wild_cat': list(data_dir_query.glob('wild_cat/*')),
    'windsor_chair': list(data_dir_query.glob('windsor_chair/*')),
    'wrench': list(data_dir_query.glob('wrench/*')),
    'yin_yang': list(data_dir_query.glob('yin_yang/*')),
    'crocodile': list(data_dir_query.glob('crocodile/*')),
    'trilobite': list(data_dir_query.glob('trilobite/*'))
}
object_labels_dict_q = {
    #'BACKGROUND_Google': 0,
     'Faces': 1,
     'Faces_easy': 2,
     'Leopards': 3,
     'Motorbikes': 4,
     'accordion': 5,
     'airplanes': 6,
     'anchor': 7,
     'ant': 8,
     'barrel': 9,
     'bass': 10,
     'beaver': 11,
     'binocular': 12,
     'bonsai': 13,
     'brain': 14,
     'brontosaurus': 15,
     'buddha': 16,
     'butterfly': 17,
     'camera': 18,
     'cannon': 19,
     'car_side': 20,
     'ceiling_fan': 21,
     'cellphone': 22,
     'chair': 23,
     'chandelier': 24,
     'cougar_body': 25,
     'cougar_face': 26,
     'crab': 27,
     'crayfish': 28,
     'crocodile_head': 29,
     'cup': 30,
     'dalmatian': 31,
     'dollar_bill': 32,
     'dolphin': 33,
     'dragonfly': 34,
     'electric_guitar': 35,
     'elephant': 36,
     'emu': 37,
     'euphonium': 38,
     'ewer': 39,
     'ferry': 40,
     'flamingo': 41,
     'flamingo_head': 42,
     'garfield': 43,
     'gerenuk': 44,
     'gramophone': 45,
     'grand_piano': 46,
     'hawksbill': 47,
     'headphone': 48,
     'hedgehog': 49,
     'helicopter': 50,
     'ibis': 51,
     'inline_skate': 52,
     'joshua_tree': 53,
     'kangaroo': 54,
     'ketch': 55,
     'lamp': 56,
     'laptop': 57,
     'llama': 58,
     'lobster': 59,
     'lotus': 60,
     'mandolin': 61,
     'mayfly': 62,
     'menorah': 63,
     'metronome': 64,
     'minaret': 65,
     'nautilus': 66,
     'octopus': 67,
     'okapi': 68,
     'pagoda': 69,
     'panda': 70,
     'pigeon': 71,
     'pizza':72,
     'platypus': 73,
     'pyramid': 74,
     'revolver': 75,
     'rhino': 76,
     'rooster': 77,
     'saxophone': 78,
     'schooner': 79,
     'scissors': 80,
     'scorpion': 81,
     'sea_horse': 82,
     'snoopy': 83,
     'soccer_ball': 84,
     'stapler': 85,
     'starfish': 86,
     'stegosaurus': 87,
     'stop_sign': 88,
     'strawberry': 89,
     'sunflower': 90,
     'tick': 91,
     'umbrella': 92,
     'watch': 93,
     'water_lilly': 94,
     'wheelchair': 95,
     'wild_cat': 96,
     'windsor_chair': 97,
     'wrench': 98,
     'yin_yang': 99,
     'crocodile':100,
     'trilobite':101, 
}
#class_to_numeric = {class_label: i for i, class_label in enumerate(set(class_labels))}
for object_category,images in object_categories_dict_q.items():
    for image in images:
        dataset_labels_query.append(object_labels_dict_q[object_category])


In [ ]:
#resize query images 
X_q, y_q = [], []
for object_category,filenames_query_images in object_categories_dict_q.items():
    for filename in filenames_query_images:
        img = cv2.imread(str(filename))  # Convert Path object to string
        resized_img = cv2.resize(img, (224, 224))
        X_q.append(resized_img)
        y_q.append(object_labels_dict_q[object_category])

In [ ]:
X_q = np.array(X_q) 
y_q = np.array(y_q)

In [ ]:
x_q = X_q / 255.0  # Normalize pixel values to the range [0, 1]

In [ ]:
len(x_q)

In [ ]:
print("x_q dimensions:", x_q.shape)
print("y_q dimensions:", y_q.shape)

In [ ]:
y_q

In [ ]:
#extract features of query images 
features_query_model1 = feature_extraction_model1.predict(x_q)
features_query_model2 = feature_extraction_model2.predict(x_q)
# Concatenate both concatenated feature vectors
extracted_features_query_images = Concatenate()([features_query_model1,features_query_model2])



In [ ]:
# Assign labels to the extracted features
features_with_labels_q = list(zip(extracted_features_query_images, y_q))

In [ ]:
# Example usage: Iterating over features with their labels
for feature, label in features_with_labels_q:
    # Access the feature vector and its corresponding label
    print("Feature shape:", feature.shape)
    print("Label:", label)
    # Perform any operations using the feature and label

In [ ]:
# Reshape the extracted features array
features_query_images = tf.reshape(extracted_features_query_images, (len(extracted_features_query_images), -1))
print("Reshaped features shape:", features_query_images.shape)


In [ ]:
#Assuming features is your tensor
num_images_q, num_features_per_image_q = features_query_images.shape

# Printing the values
print("Number of images =", num_images_q)
print("Number of features per image =", num_features_per_image_q)

# Perform Retrieval

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree
import timeit

def ret_images_knn(query_features, k):
    # Fit a Nearest Neighbors model
    nn_model = NearestNeighbors(n_neighbors=k, algorithm='auto')
    nn_model.fit(extracted_features)

    # Query for the nearest neighbors
    start_time = time.time()
    distances, top_indices = nn_model.kneighbors(query_features)
    end_time = time.time()
    retrieval_time = end_time - start_time
    print(f"Retrieval Time: {retrieval_time:.4f} seconds")
    print("Distances", distances) 
    # Plot the query image
    plt.figure(figsize=(15, 2))
    plt.subplot(1, 6, 1)
    plt.imshow(x_q[query_index])
    plt.title("Query Image")

    # Plot the top 5 similar images
    retrieved_images_with_labels = []
    retrieved_labels = []
    for i, (index, distance) in enumerate(zip(top_indices[0], distances[0]), start=2):
        plt.subplot(1, k+1, i)
        plt.imshow(x[index])
        plt.title(f"Similar Image {i-1}\nDistance: {distance:.4f}")
        # Append the retrieved image and its label to the list
        retrieved_images_with_labels.append((x[index], y[index]))
        retrieved_labels.append(y[index])
    #plt.tight_layout()    
    plt.show()
    return retrieved_images_with_labels,retrieved_labels,top_indices,retrieval_time


    plt.show()

In [ ]:
def calculate_metrics(query_labels, retrieved_labels,k):
    if len(query_labels) != 1:
        raise ValueError("The query should have a single label.")

    query_label = query_labels[0]
    # Ensure retrieved_labels is a list
    if not isinstance(retrieved_labels, list):
        retrieved_labels = [retrieved_labels]
        
    # Ensure k does not exceed the length of retrieved_labels
    k = min(k, len(retrieved_labels))    

    #for k in k_values:
    top_k_retrieved = retrieved_labels[:k]
    
    # Calculate true positives
    true_positive = sum(1 for label in top_k_retrieved if label == query_label)
    
    # Calculate false positives
    false_positive = k - true_positive
    
    # Calculate false negatives
    false_negative = 1 if true_positive == 0 else 0  # There is a false negative if no true positive
    
    # Calculate true negatives
    true_negative = k - true_positive  # Assuming binary classification
    
    # Calculate accuracy at k
    accuracy_at_k = (true_positive + true_negative) / k
    
    # Calculate precision at k
    precision_at_k = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0.0
    
    # Calculate recall at k
    recall_at_k = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0.0
    
    # Calculate F1 score at k
    f_measure_at_k = 2 * (precision_at_k * recall_at_k) / (precision_at_k + recall_at_k) if (precision_at_k + recall_at_k) > 0 else 0.0
    
    
    return precision_at_k, recall_at_k, accuracy_at_k, f_measure_at_k



In [ ]:
import pandas as pd

def store_metrics_to_excel(metrics_data, retrieval_methods, filename):
    # Create a Pandas Excel writer using XlsxWriter as the engine
    writer = pd.ExcelWriter(filename, engine='xlsxwriter')
    
    # Loop over the retrieval methods
    for method in retrieval_methods:
        # Convert the dictionary to a DataFrame
        df = pd.DataFrame(metrics_data[method])
        
        # Write each DataFrame to a separate sheet
        df.to_excel(writer, sheet_name=method, index=False)
    
    # Close the Pandas Excel writer and save the file
    writer.save()

In [ ]:
#Retrieving top 20 images
k = 5

# Initialize an empty dictionary to store metrics data
all_metrics_data = {'euclidean': {}}

# Loop over all queries
for i in range(len(x_q)):
    query_labels = []
    query_index = i
    print("Query image",i)
    query_features = tf.reshape(features_query_images[query_index], (1, -1))
    print(query_features.shape)
    query_label = y_q[query_index]
    print(query_label)
    query_labels.append(query_label)
    
    # Perform retrieval using KNN
    retrieved_images_with_labels, retrieved_labels, top_indices,retrieval_time = ret_images_knn(query_features, k)
    precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels, k)
    all_metrics_data['euclidean'][f"Query {i + 1}"] = [precision, recall, f1, accuracy, retrieval_time]
    print("Query labels", query_labels,"\n")
    print("Retrieved Labels", retrieved_labels,"\n")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)
    
# Store all metrics data to an Excel file
store_metrics_to_excel(all_metrics_data, ['euclidean'], 'metrics_concat_fusion_mobilnetandeffB0_results_caltech101_retrieving_using_KNN_images.xlsx')
    

In [ ]:
#Retrieving top 20 images
k = 30

# Initialize an empty dictionary to store metrics data
all_metrics_data = {'KNN': {}, 'Cosine Similarity': {}, 'Euclidean Distance': {}}

# Loop over all queries
for i in range(len(x_q)):
    query_labels = []
    query_index = i
    print("Query image",i)
    query_features = tf.reshape(features_query_images[query_index], (1, -1))
    print(query_features.shape)
    query_label = y_q[query_index]
    print(query_label)
    query_labels.append(query_label)
    
    # Retrieve images using cosine similarity
    retrieved_images_with_labels, retrieved_labels = ret_images_cosine_similarity(query_features,k)
    precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels, k)
    all_metrics_data['Cosine Similarity'][f"Query {i + 1}"] = [precision, recall, f1, accuracy]
    print("Query labels", query_labels,"\n")
    print("Retrieved Labels", retrieved_labels,"\n")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)

    # Retrieve images using euclidean distance
    retrieved_images_with_labels, retrieved_labels, distances = ret_images_euclidean_distance(query_features,k)
    precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels, k)
    all_metrics_data['Euclidean Distance'][f"Query {i + 1}"] = [precision, recall, f1, accuracy]
    print("Query labels", query_labels,"\n")
    print("Retrieved Labels", retrieved_labels,"\n")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)

    # Perform retrieval using KNN
    retrieved_images_with_labels, retrieved_labels, top_indices = ret_images_knn(query_features, k)
    precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels, k)
    all_metrics_data['KNN'][f"Query {i + 1}"] = [precision, recall, f1, accuracy]
    print("Query labels", query_labels,"\n")
    print("Retrieved Labels", retrieved_labels,"\n")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)
        
# Store all metrics data to an Excel file
store_metrics_to_excel(all_metrics_data, ['KNN', 'Cosine Similarity', 'Euclidean Distance'], 'metrics_concat_fusion_mobilnetandeffB0_results_corel1k_retrieving_top30_images.xlsx')


In [ ]:
#Retrieving top 20 images
k = 25

# Initialize an empty dictionary to store metrics data
all_metrics_data = {'KNN': {}, 'Cosine Similarity': {}, 'Euclidean Distance': {}}

# Loop over all queries
for i in range(len(x_q)):
    query_labels = []
    query_index = i
    print("Query image",i)
    query_features = tf.reshape(features_query_images[query_index], (1, -1))
    print(query_features.shape)
    query_label = y_q[query_index]
    print(query_label)
    query_labels.append(query_label)
    
    # Retrieve images using cosine similarity
    retrieved_images_with_labels, retrieved_labels = ret_images_cosine_similarity(query_features,k)
    precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels, k)
    all_metrics_data['Cosine Similarity'][f"Query {i + 1}"] = [precision, recall, f1, accuracy]
    print("Query labels", query_labels,"\n")
    print("Retrieved Labels", retrieved_labels,"\n")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)

    # Retrieve images using euclidean distance
    retrieved_images_with_labels, retrieved_labels, distances = ret_images_euclidean_distance(query_features,k)
    precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels, k)
    all_metrics_data['Euclidean Distance'][f"Query {i + 1}"] = [precision, recall, f1, accuracy]
    print("Query labels", query_labels,"\n")
    print("Retrieved Labels", retrieved_labels,"\n")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)

    # Perform retrieval using KNN
    retrieved_images_with_labels, retrieved_labels, top_indices = ret_images_knn(query_features, k)
    precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels, k)
    all_metrics_data['KNN'][f"Query {i + 1}"] = [precision, recall, f1, accuracy]
    print("Query labels", query_labels,"\n")
    print("Retrieved Labels", retrieved_labels,"\n")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)
        
# Store all metrics data to an Excel file
store_metrics_to_excel(all_metrics_data, ['KNN', 'Cosine Similarity', 'Euclidean Distance'], 'metrics_concat_fusion_mobilnetandeffB0_results_corel1k_retrieving_top25_images.xlsx')


In [ ]:
#Retrieving top 20 images
k = 20

# Initialize an empty dictionary to store metrics data
all_metrics_data = {'KNN': {}, 'Cosine Similarity': {}, 'Euclidean Distance': {}}

# Loop over all queries
for i in range(len(x_q)):
    query_labels = []
    query_index = i
    print("Query image",i)
    query_features = tf.reshape(features_query_images[query_index], (1, -1))
    print(query_features.shape)
    query_label = y_q[query_index]
    print(query_label)
    query_labels.append(query_label)
    
    # Retrieve images using cosine similarity
    retrieved_images_with_labels, retrieved_labels = ret_images_cosine_similarity(query_features,k)
    precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels, k)
    all_metrics_data['Cosine Similarity'][f"Query {i + 1}"] = [precision, recall, f1, accuracy]
    print("Query labels", query_labels,"\n")
    print("Retrieved Labels", retrieved_labels,"\n")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)

    # Retrieve images using euclidean distance
    retrieved_images_with_labels, retrieved_labels, distances = ret_images_euclidean_distance(query_features,k)
    precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels, k)
    all_metrics_data['Euclidean Distance'][f"Query {i + 1}"] = [precision, recall, f1, accuracy]
    print("Query labels", query_labels,"\n")
    print("Retrieved Labels", retrieved_labels,"\n")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)

    # Perform retrieval using KNN
    retrieved_images_with_labels, retrieved_labels, top_indices = ret_images_knn(query_features, k)
    precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels, k)
    all_metrics_data['KNN'][f"Query {i + 1}"] = [precision, recall, f1, accuracy]
    print("Query labels", query_labels,"\n")
    print("Retrieved Labels", retrieved_labels,"\n")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)
        
# Store all metrics data to an Excel file
store_metrics_to_excel(all_metrics_data, ['KNN', 'Cosine Similarity', 'Euclidean Distance'], 'metrics_concat_fusion_mobilnetandeffB0_results_corel1k_retrieving_top20_images.xlsx')


In [ ]:
#Retrieving top 10 images
k = 10

# Initialize an empty dictionary to store metrics data
all_metrics_data = {'KNN': {}, 'Cosine Similarity': {}, 'Euclidean Distance': {}}

# Loop over all queries
for i in range(len(x_q)):
    query_labels = []
    query_index = i
    print("Query image",i)
    query_features = tf.reshape(features_query_images[query_index], (1, -1))
    print(query_features.shape)
    query_label = y_q[query_index]
    print(query_label)
    query_labels.append(query_label)
    
    # Retrieve images using cosine similarity
    retrieved_images_with_labels, retrieved_labels = ret_images_cosine_similarity(query_features,k)
    precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels, k)
    all_metrics_data['Cosine Similarity'][f"Query {i + 1}"] = [precision, recall, f1, accuracy]
    print("Query labels", query_labels,"\n")
    print("Retrieved Labels", retrieved_labels,"\n")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)

    # Retrieve images using euclidean distance
    retrieved_images_with_labels, retrieved_labels, distances = ret_images_euclidean_distance(query_features,k)
    precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels, k)
    all_metrics_data['Euclidean Distance'][f"Query {i + 1}"] = [precision, recall, f1, accuracy]
    print("Query labels", query_labels,"\n")
    print("Retrieved Labels", retrieved_labels,"\n")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)

    # Perform retrieval using KNN
    retrieved_images_with_labels, retrieved_labels, top_indices = ret_images_knn(query_features, k)
    precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels, k)
    all_metrics_data['KNN'][f"Query {i + 1}"] = [precision, recall, f1, accuracy]
    print("Query labels", query_labels,"\n")
    print("Retrieved Labels", retrieved_labels,"\n")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)
        
# Store all metrics data to an Excel file
store_metrics_to_excel(all_metrics_data, ['KNN', 'Cosine Similarity', 'Euclidean Distance'], 'metrics_concat_fusion_mobilnetandeffB0_results_corel1k_retrieving_top10_images.xlsx')


In [ ]:
k=5

# Initialize an empty dictionary to store metrics data
all_metrics_data = {'KNN': {}, 'Cosine Similarity': {}, 'Euclidean Distance': {}}

# Loop over all queries
for i in range(len(x_q)):
    query_labels = []
    query_index = i
    print("Query image",i)
    query_features = tf.reshape(extracted_features_query_images[query_index], (1, -1))
    query_label = y_q[query_index]
    print(query_label)
    query_labels.append(query_label)
    
    # Retrieve images using cosine similarity
    retrieved_images_with_labels, retrieved_labels = ret_images_cosine_similarity(query_features,k)
    precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels, k)
    all_metrics_data['Cosine Similarity'][f"Query {i + 1}"] = [precision, recall, f1, accuracy]
    print("Query labels", query_labels,"\n")
    print("Retrieved Labels", retrieved_labels,"\n")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)

    # Retrieve images using euclidean distance
    retrieved_images_with_labels, retrieved_labels, distances = ret_images_euclidean_distance(query_features,k)
    precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels, k)
    all_metrics_data['Euclidean Distance'][f"Query {i + 1}"] = [precision, recall, f1, accuracy]
    print("Query labels", query_labels,"\n")
    print("Retrieved Labels", retrieved_labels,"\n")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)

    # Perform retrieval using KNN
    retrieved_images_with_labels, retrieved_labels, top_indices = ret_images_knn(query_features, k)
    precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels, k)
    all_metrics_data['KNN'][f"Query {i + 1}"] = [precision, recall, f1, accuracy]
    print("Query labels", query_labels,"\n")
    print("Retrieved Labels", retrieved_labels,"\n")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)
        
# Store all metrics data to an Excel file
store_metrics_to_excel(all_metrics_data, ['KNN', 'Cosine Similarity', 'Euclidean Distance'], 'metrics_concat_fusion_mobilnetandeffB0_results_caltech101.xlsx')


# End of Code

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib
import matplotlib.pyplot as plt
#Retrieve images using cosine similarity
def ret_images_cosine_similarity(query_features):
    similarity_scores = cosine_similarity(query_features, embedded_features)
    #print(similarity_scores)
    # Get the indices of the top 5 most similar images (excluding the query image itself)
    top_indices = np.argsort(similarity_scores[0])[::-1][1:6]
    #print(top_indices)
    # Plot the query image
    plt.figure(figsize=(10, 2))
    plt.subplot(1, 6, 1)
    plt.imshow(x[query_index])
    plt.title("Query Image")

    # Plot the top 5 similar images
    retrieved_images_with_labels = []
    retrieved_labels = []
    for i, index in enumerate(top_indices, start=2):
        print("Index:", index)
        plt.subplot(1, 6, i)
        plt.imshow(x[index])
        plt.title(f"Similar Image {i-1}")
        # Append the retrieved image and its label to the list
        retrieved_images_with_labels.append((x[index], y[index]))
    
        retrieved_labels.append(y[index])
    plt.tight_layout()    
    plt.show()
    return retrieved_images_with_labels,retrieved_labels

In [ ]:
#retrieve images using cosine similarity
retrieved_images_with_labels,retrieved_labels = ret_images_cosine_similarity(query_features)

In [ ]:
retrieved_labels

In [ ]:
print(type(retrieved_labels))

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

def ret_images_euclidean_distance(query_features):
    # Calculate Euclidean distances
    distances = euclidean_distances(query_features, embedded_features)
    # Get the indices of the top 5 most similar images
    top_indices = np.argsort(distances[0])[:5]
    #print(top_distances)
    # Plot the query image
    plt.figure(figsize=(10, 2))
    plt.subplot(1, 6, 1)
    plt.imshow(x[query_index])
    plt.title("Query Image")

    # Plot the top 5 similar images
    retrieved_images_with_labels = []
    retrieved_labels = []
    for i, index in enumerate(top_indices, start=2):
        plt.subplot(1, 6, i)
        plt.imshow(x[index])
        plt.title(f"Similar Image {i-1}")
        # Append the retrieved image and its label to the list
        retrieved_images_with_labels.append((x[index], y[index]))
        retrieved_labels.append(y[index])
    plt.tight_layout()    
    plt.show()
    return retrieved_images_with_labels,retrieved_labels,distances


In [ ]:
retrieved_images_with_labels,retrieved_labels,distances = ret_images_euclidean_distance(query_features)

In [ ]:
retrieved_labels

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree

def ret_images_knn(query_features, k=5):
    # Fit a Nearest Neighbors model
    nn_model = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
    nn_model.fit(embedded_features)

    # Query for the nearest neighbors
    distances, top_indices = nn_model.kneighbors(query_features)
    print(distances) 
    # Plot the query image
    plt.figure(figsize=(15, 2))
    plt.subplot(1, 6, 1)
    plt.imshow(x[query_index])
    plt.title("Query Image")

    # Plot the top 5 similar images
    retrieved_images_with_labels = []
    retrieved_labels = []
    for i, (index, distance) in enumerate(zip(top_indices[0], distances[0]), start=2):
        plt.subplot(1, 6, i)
        plt.imshow(x[index])
        plt.title(f"Similar Image {i-1}\nDistance: {distance:.4f}")
        # Append the retrieved image and its label to the list
        retrieved_images_with_labels.append((x[index], y[index]))
        retrieved_labels.append(y[index])
    #plt.tight_layout()    
    plt.show()
    return retrieved_images_with_labels,retrieved_labels,top_indices


    plt.show()


In [ ]:
# Perform retrieval using KNN
retrieved_images_with_labels,retrieved_labels,top_indices = ret_images_knn(query_features, k=5)

In [ ]:
retrieved_labels

In [ ]:
query_labels

In [ ]:
type(query_labels)

In [ ]:
type(retrieved_labels)

In [ ]:
def calculate_metrics(query_labels, retrieved_labels,k):
    if len(query_labels) != 1:
        raise ValueError("The query should have a single label.")

    query_label = query_labels[0]
    # Ensure retrieved_labels is a list
    if not isinstance(retrieved_labels, list):
        retrieved_labels = [retrieved_labels]
        
    # Ensure k does not exceed the length of retrieved_labels
    k = min(k, len(retrieved_labels))    

    #for k in k_values:
    top_k_retrieved = retrieved_labels[:k]
    
    # Calculate true positives
    true_positive = sum(1 for label in top_k_retrieved if label == query_label)
    
    # Calculate false positives
    false_positive = k - true_positive
    
    # Calculate false negatives
    false_negative = 1 if true_positive == 0 else 0  # There is a false negative if no true positive
    
    # Calculate true negatives
    true_negative = k - true_positive  # Assuming binary classification
    
    # Calculate accuracy at k
    accuracy_at_k = (true_positive + true_negative) / k
    
    # Calculate precision at k
    precision_at_k = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0.0
    
    # Calculate recall at k
    recall_at_k = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0.0
    
    # Calculate F1 score at k
    f_measure_at_k = 2 * (precision_at_k * recall_at_k) / (precision_at_k + recall_at_k) if (precision_at_k + recall_at_k) > 0 else 0.0
    
    
    return precision_at_k, recall_at_k, accuracy_at_k, f_measure_at_k



In [ ]:
k = 5
# Example usage
precision, recall, accuracy, f1 = calculate_metrics(query_labels, retrieved_labels,k)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Accuracy:", accuracy)


In [ ]:
import numpy as np

def calculate_relevance_scores(distances, epsilon=1e-6):
    # Convert distances to relevance scores
    relevance_scores = 1 / (np.array(distances) + epsilon)
    return relevance_scores

In [ ]:
print(distances)
relevance_scores = calculate_relevance_scores(distances)
relevance_scores

In [ ]:
def calculate_ndcg(relevance_scores, k=None):
    # Sort relevance scores in descending order
    sorted_scores = np.sort(relevance_scores)[::-1]
    if k is None:
        k = len(relevance_scores)

    # Calculate DCG (Discounted Cumulative Gain)
    dcg = np.sum((2 ** sorted_scores - 1) / np.log2(np.arange(2, len(sorted_scores) + 2)))

    # Calculate IDCG (Ideal Discounted Cumulative Gain)
    num_rel = min(len(relevance_scores), k)
    ideal_scores = np.sort(1 / np.arange(1, num_rel + 1))[::-1]
    idcg = np.sum((2 ** ideal_scores - 1) / np.log2(np.arange(2, len(ideal_scores) + 2)))
    
    # Check if IDCG is zero to avoid division by zero
    if idcg == 0:
        return 0

    # Calculate NDCG (Normalized Discounted Cumulative Gain)
    ndcg = dcg / idcg
    return ndcg

In [ ]:
ndcg = calculate_ndcg(relevance_scores)
print("NDCG:", ndcg)